In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
%cd "/content/gdrive/My Drive/Colab Notebooks/MRNet/"

Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/MRNet


In [2]:

import os
import numpy as np
import glob
import csv
import keras
from partition import partition
from data_generator import MRNet_data_generator
from keras.layers import Conv2D,Input, MaxPool2D, AveragePooling2D, Dropout, Dense, Flatten


from inception_model_generator import MRNet_inc_model
import tensorflow as tf


Using TensorFlow backend.


In [0]:
b_size = 1

data_path = "/content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0"

p, l = partition(data_path)
data_gen = MRNet_data_generator(data_path, p, l, batch_size=1,  train=False,shuffle=True,scale_to=(299,299), model="inc",aug_size=1)


In [4]:

os.chdir('/content/gdrive/My Drive/Colab Notebooks/MRNet')
model, cp_callback = MRNet_inc_model(["abnormal", "axial"])


pos = 0
for e in p['train']['axial']:
  if l[e]['abnormal'] == 1:
    pos += 1
total = len(p['train']['axial'])
neg = total - pos
weight_for_0 = (1 / neg)*(total)/2.0 
weight_for_1 = (1 / pos)*(total)/2.0
class_weight = {0: weight_for_0, 1: weight_for_1}

model.fit(x=data_gen, epochs=10, verbose=True, callbacks=[cp_callback],shuffle=False, use_multiprocessing=True, workers=6, class_weight=class_weight)
model.save_weights("training_inc_1/w.h5")

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mr_net_inception_layer_1 (MR (None, 1)                 23066273  
Total params: 23,066,273
Trainable params: 23,066,273
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
120/120 [==============================] - 240s 2s/step - loss: 5.2158 - tp: 39.1833 - fp: 36.3417 - tn: 70.7000 - fn: 51.2333 - accuracy: 0.5453 - precision: 0.4782 - recall: 0.4164 - auc: 0.4414

Epoch 00001: saving model to training_inception/abnormal/axialweights01.hdf5


OSError: ignored

In [0]:
b_size = 1

data_path = "/content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0"
p, l = partition(data_path)


In [0]:

valid_gen = MRNet_data_generator(data_path, p, l, batch_size=b_size, scale_to=(299,299),train=False, shuffle=False,model="inc",aug_size=0)
print(valid_gen.__len__());

In [0]:
model, cp_callback = MRNet_inc_model(["abnormal", "axial"])
os.chdir('/content/gdrive/My Drive/Colab Notebooks/MRNet')

model.load_weights('training_inception/abnormal/axial/cp.ckpt');


In [0]:
d = model.evaluate(x=valid_gen,verbose=1,use_multiprocessing=True,workers=6);
print(d)


In [0]:
model.metrics_names

In [0]:

d = model.predict(x=valid_gen,verbose=1,use_multiprocessing=True,workers=6)
print(d)